In [1]:
%pylab
%reload_ext autoreload
%autoreload 2


import pandas as pd

Using matplotlib backend: Qt5Agg
Populating the interactive namespace from numpy and matplotlib


In [2]:
fashion_mnist = pd.read_csv('feat7k.csv')

In [3]:
fashion_mnist.shape

(7653, 4097)

In [31]:
import os 
from natsort import natsorted

names = natsorted(os.listdir('/home/parawr/Projects/clusterFacadeData/superFacade'))
names_to_class = {ii:o for ii, o in enumerate(names)}

names_to_class[10] = 'Switzerlan'

In [5]:
{'Austria': 0,
 'Belgium': 1,
 'China': 2,
 'Denmark': 3,
 'France': 4,
 'Germany': 5,
 'Greece': 6,
 'Italy': 7,
 'Netherlands': 8,
 'Romania': 9,
 'Switzerland': 10,
 'UK': 11,
 'USA_Chicago': 12,
 'USA_Dallas': 13,
 'USA_Hawaii': 14,
 'USA_LosAngeles': 15,
 'USA_NewYork': 16}

{'Austria': 0,
 'Belgium': 1,
 'China': 2,
 'Denmark': 3,
 'France': 4,
 'Germany': 5,
 'Greece': 6,
 'Italy': 7,
 'Netherlands': 8,
 'Romania': 9,
 'Switzerland': 10,
 'UK': 11,
 'USA_Chicago': 12,
 'USA_Dallas': 13,
 'USA_Hawaii': 14,
 'USA_LosAngeles': 15,
 'USA_NewYork': 16}

In [37]:
import pickle 
with open('id_to_name7k.pickle', 'rb') as fd:
    id_to_name = pickle.load(fd)
    
id_to_class = []
for ii, name in enumerate(id_to_name):
    for jj, class_name in names_to_class.items():
        if class_name in name:
            id_to_class.append(jj)

In [33]:
len(id_to_class)

7653

In [34]:
set(range(7653)).difference(set(id_to_class.keys()))

set()

In [30]:
id_to_name[5725]

'Switzerlan_Bern_way97246574_wall_3_2_n3Jx5nKwFqvde_0OB7l2SA_VP_0_1_FId_749_.jpg'

In [35]:
nrows, ncols = fashion_mnist.shape
nrows, ncols

(7653, 4097)

In [38]:
id_to_class = id_to_class[:nrows]
len(id_to_class)

7653

In [39]:
import umap
import numpy as np
import pandas as pd
import requests
import os
import datashader as ds
import datashader.utils as utils
import datashader.transfer_functions as tf

import matplotlib.pyplot as plt
import seaborn as sns
import time

sns.set(context="paper", style="white")

source_df = pd.read_csv('feat7k.csv')

data = source_df.iloc[:, 1:4095].values.astype(np.float32)
target = id_to_class

In [40]:
pal = [
    '#FFFFFF',
    '#000000',
    '#D70000',
    '#8C3CFF',
    '#028800',
    '#00ACC7',
    '#98FF00',
    '#FF7FD1',
    '#6C004F',
    '#FFA530',
    '#00009D',
    '#867068',
    '#004942',
    '#4F2A00',
    '#00FDCF',
    '#BCB7FF',
    '#95B47A',
    ]
color_key = {str(d):c.lower() for d,c in enumerate(pal)}
color_key

{'0': '#ffffff',
 '1': '#000000',
 '10': '#00009d',
 '11': '#867068',
 '12': '#004942',
 '13': '#4f2a00',
 '14': '#00fdcf',
 '15': '#bcb7ff',
 '16': '#95b47a',
 '2': '#d70000',
 '3': '#8c3cff',
 '4': '#028800',
 '5': '#00acc7',
 '6': '#98ff00',
 '7': '#ff7fd1',
 '8': '#6c004f',
 '9': '#ffa530'}

In [41]:
reducer = umap.UMAP(random_state=42)
start = time.time()
embedding = reducer.fit_transform(data)
print(time.time() - start)
np.savez('embedding4096.npz', embedding)

293.666139125824


In [42]:
np.unique(target)

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16])

In [15]:
df = pd.DataFrame(embedding, columns=('x', 'y'))
df['class'] = pd.Series([str(x) for x in target], dtype="category")

In [29]:
cvs = ds.Canvas(plot_width=400, plot_height=400)
agg = cvs.points(df, 'x', 'y', ds.count_cat('class'))
img = tf.shade(agg, color_key=color_key, how='eq_hist')


utils.export_image(img, filename='feat7k', background='white')

image = plt.imread('feat7k.png')
fig, ax = plt.subplots(figsize=(6, 6))
plt.imshow(image)
plt.setp(ax, xticks=[], yticks=[])
plt.title("Fashion MNIST data embedded\n"
          "into two dimensions by UMAP\n"
          "visualised with Datashader",
          fontsize=12)

plt.show()

In [46]:
sc = plt.scatter(embedding[:, 0], embedding[:, 1], c=target, cmap=plt.cm.tab20)
cbar = plt.colorbar(sc)

cbar.set_ticks(range(17))
cbar.set_ticklabels(list(names_to_class.values()))


In [47]:
len(list(names_to_class.values())+['x', 'x', 'x'])

20